In [2]:
import pandas as pd


In [32]:
aws_working = pd.read_csv('aws_working_pass.tsv',sep='\t', header=None)
aws_working.rename(columns={0:'s3-working'},inplace=True)
methylation_summary = pd.read_csv('methylation_summary.tsv',sep='\t')

In [33]:
methylation_summary.head()

,file_name,MM_tag,ML_tag,all_kinetics_flag,keep_kinetics_flag,hifi_kinetics_tag,PP_PRIMROSE,fi_tag,ri_tag,fp_tag,rp_tag
0,m84046_230716_051716_s3.hifi_reads.bc2086.bam,True,True,False,True,False,True,True,True,True,True
1,m84046_230716_054822_s4.hifi_reads.bc2086.bam,True,True,False,True,False,True,True,True,True,True
2,m84046_230718_223822_s1.hifi_reads.bc2086.bam,True,True,False,True,False,True,True,True,True,True
3,m54329U_220825_174247-bc2012.5mc.hifi_reads.bam,True,True,True,True,False,True,True,True,True,True
4,m54329U_220827_143814-bc2050.5mc.hifi_reads.bam,True,True,True,True,False,True,True,True,True,True


In [34]:
aws_working['sample_id'] = [sample.split('/')[-4] for sample in aws_working['s3-working'].tolist()]
aws_working['file_name'] = [sample.split('/')[-1] for sample in aws_working['s3-working'].tolist()]

In [35]:
meth_df = pd.merge(methylation_summary, aws_working, on='file_name',how='inner')

In [36]:
meth_kinetics_df = meth_df[meth_df.loc[:, ['fi_tag','ri_tag','fp_tag','rp_tag']].all(axis=1) == True]

In [37]:
meth_to_run = meth_kinetics_df[meth_kinetics_df.loc[:, ['MM_tag','ML_tag']].all(axis=1) == False]

In [38]:
meth_to_run = meth_to_run[meth_to_run['file_name'].str.contains('.hifi_reads.bam')]

In [39]:
meth_to_run['sample_id'] = meth_to_run['sample_id'] + '-' + meth_to_run['file_name']

In [40]:
meth_to_run.head()

,file_name,MM_tag,ML_tag,all_kinetics_flag,keep_kinetics_flag,hifi_kinetics_tag,PP_PRIMROSE,fi_tag,ri_tag,fp_tag,rp_tag,s3-working,sample_id
87,m64043_210205_190424.hifi_reads.bam,False,False,True,False,False,False,True,True,True,True,s3://human-pangenomics/working/HPRC/HG00423/ra...,HG00423-m64043_210205_190424.hifi_reads.bam
89,m64043_210207_011920.hifi_reads.bam,False,False,True,False,False,False,True,True,True,True,s3://human-pangenomics/working/HPRC/HG00423/ra...,HG00423-m64043_210207_011920.hifi_reads.bam
91,m64043_210209_184051.hifi_reads.bam,False,False,True,False,False,False,True,True,True,True,s3://human-pangenomics/working/HPRC/HG00423/ra...,HG00423-m64043_210209_184051.hifi_reads.bam
93,m64043_210211_005516.hifi_reads.bam,False,False,True,False,False,False,True,True,True,True,s3://human-pangenomics/working/HPRC/HG00423/ra...,HG00423-m64043_210211_005516.hifi_reads.bam
99,m64043_210702_183246.hifi_reads.bam,False,False,True,False,False,False,True,True,True,True,s3://human-pangenomics/working/HPRC/HG00544/ra...,HG00544-m64043_210702_183246.hifi_reads.bam


In [41]:
meth_to_run = meth_to_run[['sample_id','s3-working']]
meth_to_run.rename(columns={'s3-working':'input_bams'},inplace=True)


In [42]:
meth_to_run['input_bams'] = [f"{[input_bam]}" for input_bam in meth_to_run['input_bams'].tolist()]

In [43]:
meth_to_run.drop_duplicates()

,sample_id,input_bams
87,HG00423-m64043_210205_190424.hifi_reads.bam,['s3://human-pangenomics/working/HPRC/HG00423/...
89,HG00423-m64043_210207_011920.hifi_reads.bam,['s3://human-pangenomics/working/HPRC/HG00423/...
91,HG00423-m64043_210209_184051.hifi_reads.bam,['s3://human-pangenomics/working/HPRC/HG00423/...
93,HG00423-m64043_210211_005516.hifi_reads.bam,['s3://human-pangenomics/working/HPRC/HG00423/...
99,HG00544-m64043_210702_183246.hifi_reads.bam,['s3://human-pangenomics/working/HPRC/HG00544/...
...,...,...
787,HG04228-m54329U_210507_224951-bc1008_BAK8A_OA....,['s3://human-pangenomics/working/HPRC/HG04228/...
788,HG04228-m54329U_210725_100455.hifi_reads.bam,['s3://human-pangenomics/working/HPRC/HG04228/...
789,HG04228-m64076_210617_050849.hifi_reads.bam,['s3://human-pangenomics/working/HPRC/HG04228/...
790,HG04228-m64076_210618_160639.hifi_reads.bam,['s3://human-pangenomics/working/HPRC/HG04228/...


In [44]:
meth_to_run.to_csv('primrose/primrose_sample_data.csv',index=False)

In [45]:
meth_to_run.shape

(171, 2)